In [ ]:
import pandas as pd

### 1. Import Datasets
The data is originally provided as .txt file. I read the .txt file in excel and saved it as .csv

In [24]:
df_neg = pd.read_csv('train_neg.csv', sep="\t")
df_neg.columns = ["reviewText"]
df_neg["rating"] = 0

In [25]:
df_neg.shape

(1500, 2)

In [26]:
df_neg.head()

,reviewText,rating
0,Came in a brown half liter bottle with a good ...,0
1,"12 oz bottle poured into a pint glass, got a b...",0
2,Appearance: Pours a rich amber-red color with ...,0
3,I can't imagine a worse winter warmer. Surely ...,0
4,"12 oz bottle best by 8/9/11, poured into a pin...",0


In [31]:
df_pos = pd.read_csv('train_pos.csv', sep="\t")
df_pos.columns = ["reviewText"]
df_pos["rating"] = 1

In [32]:
df_pos.shape

(1500, 2)

In [33]:
df_pos.head()

,reviewText,rating
0,Too imaptient- poured straight from fridge int...,1
1,Kinda surprised I haven't reviewed this long a...,1
2,Thanks goes out to goryshkewych and roopy40 fo...,1
3,A - Pours a ruby red with two fingers of head....,1
4,"from the bottle, poured crimson red with a nic...",1


In [34]:
df_test = pd.read_csv('test.csv', sep="\t")
df_test.columns = ["reviewText"]
#nur beim ersten Durchlauf:
#df_pos["rating"] = None

In [35]:
df_test.head()

,reviewText
0,12oz bottle; bottled-on date Sept '10.\t\tPour...
1,Poured from a 22 oz. bottle into a standard pi...
2,Pours a cloudy honey like color with a thin la...
3,750 ml bottle courtesy of myspoonistoobig. Tha...
4,A: Hazy amber in color with a half finger head...


In [36]:
df_all = df_neg.append(df_pos)

In [37]:
df_all.head(5)

,reviewText,rating
0,Came in a brown half liter bottle with a good ...,0
1,"12 oz bottle poured into a pint glass, got a b...",0
2,Appearance: Pours a rich amber-red color with ...,0
3,I can't imagine a worse winter warmer. Surely ...,0
4,"12 oz bottle best by 8/9/11, poured into a pin...",0


In [38]:
df_all.shape

(3000, 2)

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


In [41]:
#kann mit X_train, X_test, y_train, y_test = train_test_split(df_all["reviewText"],
#                                                                df_all["rating"], 
#                                                                test_size=0.33, 
#                                                                random_state=0)
#gesplittet und auf das triningsset angewand werden. 
#Da zum Training für die Klassifikation von den ungelabelten Daten aber das gesamte 
#Traininsset verwendet wird, ist das hier nicht nötig.

X_train = df_all["reviewText"]
y_train = df_all["rating"]
X_test  = df_test["reviewText"]

In [42]:
X_train.shape

(3000,)

In [43]:
#C und gamma Werte wurden mit Hilfe einer grid search für optimal befunden.
#Random state zur Reproduktion

pipe = Pipeline([("bow", CountVectorizer(stop_words="english", token_pattern=r'\w+')),
                 ("tfidf", TfidfTransformer()),
                 ("model", SVC(kernel="rbf", C = 10, gamma = 1, random_state = 0))])

In [44]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [ ]:
#sichert das Modell

import pickle
filename = 'finalized_rbf_svm_model.sav'
pickle.dump(pipe, open(filename, 'wb'))

In [45]:
predictions = pipe.predict(X_test)

In [46]:
df_test["predicted_rating"] = predictions

In [47]:
df_test.head(20)

,reviewText,predicted_rating
0,12oz bottle; bottled-on date Sept '10.\t\tPour...,1
1,Poured from a 22 oz. bottle into a standard pi...,1
2,Pours a cloudy honey like color with a thin la...,0
3,750 ml bottle courtesy of myspoonistoobig. Tha...,1
4,A: Hazy amber in color with a half finger head...,1
5,Hakim Stout...in a bottle at Queen of Sheeba R...,1
6,Got this one from the Brewery at the last fest...,1
7,"A - Poured a pretty and clear, light straw col...",1
8,A - A slightly cloudy version of your standard...,1
9,"Purchased at Eagle River Liquor in Fulton, Ill...",1


In [48]:
df_test.to_csv(header = True, path_or_buf="SVM_prediction.csv", sep = ";")

In [19]:
#Falls classification metrics ferstgestellt werden sollen:
# print(confusion_matrix(y_test, predictions))
# print("\n")
# print(classification_report(y_test, predictions))

[[446  80]
 [ 59 405]]


             precision    recall  f1-score   support

          0       0.88      0.85      0.87       526
          1       0.84      0.87      0.85       464

avg / total       0.86      0.86      0.86       990



In [20]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, predictions)

0.8595959595959596